# So you want to automate sending emails...

In [67]:
import smtplib
from os import path
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from dotenv import dotenv_values

## Set up a class to do this

Why? Because I like OOP...

In [71]:
class EmailSender():

    def __init__(self, config):

        # Email configuration
        self.__smtp_server = config["SMTP_SERVER"]

        try:
            self.__smtp_PORT = int(config["SMTP_PORT"])
        except:
            self.__smtp_PORT = 587

        self.__sender   = config["SENDER"]
        self.__password = config["PASSWORD"]

    # Check if the SMTP server comes from microsoft
    def __isMicrosoftSMTP(self, candidate):
        return ("outlook" in candidate) or ("office365" in candidate)

    def __connection(func):
        """Decorator that controls smtp connection and disconnection
        """
        def wrapper(self, *args, **kwargs):
            
            # Connect to the SMTP server
            try:
                smtp_obj = smtplib.SMTP(self.__smtp_server, self.__smtp_PORT)

                # For microsoft... sigh
                if self.__isMicrosoftSMTP(self.__smtp_server):
                    smtp_obj.ehlo('mylowercasehost')
                    smtp_obj.starttls()
                    smtp_obj.ehlo('mylowercasehost')
                else:
                    smtp_obj.starttls()
                
                smtp_obj.login(self.__sender, self.__password)

            except Exception as e:
                raise Exception(
                    f"Something went wrong during the connection. See:\n\n{e}"
                )

            # Do something
            func(self, smtp_obj, *args, **kwargs)

            # Disconnect from the SMTP server
            smtp_obj.quit()
            
        return wrapper
    
    def __createEmail(self, receiver, subject, message, attachmentPaths=None):

        # Create the email message
        msg = MIMEMultipart()
        msg["From"] = self.__sender
        msg["To"] = receiver
        msg["Subject"] = subject
        msg.attach(MIMEText(message, "plain"))

        # Attach multiple files if provided
        if attachmentPaths:
            
            if isinstance(attachmentPaths, str):
                attachmentPaths = [attachmentPaths]

            for attachmentPath in attachmentPaths:

                filename = path.split(attachmentPath)[-1]

                with open(attachmentPath, "rb") as attachment:
                    part = MIMEApplication(
                        attachment.read(),
                        Name=filename
                    )
                    part["Content-Disposition"] = f"attachment; filename={filename}"
                    msg.attach(part)

        return msg
    
    @__connection
    def sendEmails(self, smtp_obj, messagesDict):

        for k,v in messagesDict.items():

            if not(
                {"receiver", "subject", "message"}.issubset(set(v.keys()))
            ):
                print(f"Incomplete arguments for {k}...\nSkipping...")

            msg = self.__createEmail(**v)
            
            # Send the email
            smtp_obj.send_message(msg)

            print(f"Email sent to {k}")

## Let's send emails!

In [ ]:
# Save your credentials in a .env value (see example.env)
# and feed it to the constructor to instantiate our class
emailSenderInstance = EmailSender(dotenv_values())

# Create some messages
messagesDetails = {
    "Friend 1": {
        "receiver": "something@something.com",
        "subject": "Hello world!",
        "message": "Hi friend!\n\nSent via Python",
        "attachmentPaths": "/your/path/here/myfile.file"
    },
    "Friend 2": {
        "receiver": "somethingElse@somethingElse.com",
        "subject": "Hello world again!",
        "message": "Hi friend 2!\n\nSent via Python",
        "attachmentPaths": [
            "/your/path/here/myfile1.file",
            "/your/path/here/myfile2.file"
        ]
    }
}

# Send them!
emailSenderInstance.sendEmails(messagesDetails)

Be creative! You can use pandas and other stuff to parse contacts, messages...
As long as you structure everything as shown above and feed it to our class, this will work